COVID Data SQL Queries

In [ ]:
-- Select Data That We Will Be Using

Select Location, date, total_cases, new_cases, total_deaths, population
From CovidDeaths
Where continent is not null
Order By 1,2

In [ ]:
-- Looking at Total Cases vs. Total Deaths
-- Shows the Likelihood of Dying if You Contract COVID in USA

Select Location, date, total_cases, total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
From CovidDeaths
Where location like '%states%'
Order By 1,2

In [ ]:
-- Looking at Total Cases vs. Population
-- Shows the Percentage of Population that got COVID in USA

Select Location, date, total_cases, Population, (total_cases/population)*100 as CasePercentage
From CovidDeaths
Where location like '%states%'
Order By 1,2

In [ ]:
-- Looking at Countries with Highest Infection Rate Compared to Population

Select Location, Population, MAX(total_cases) as HighestInfectionCount, MAX((total_cases/Population))*100 as PercentPopulationInfected
From CovidDeaths
Where continent is not null
Group By Location, Population
Order By PercentPopulationInfected desc

In [ ]:
-- Showing Countries with the Highest Death count per Population

Select Location, MAX(cast(Total_deaths as int)) as TotalDeathCount
From CovidDeaths
Where continent is not null
Group By Location
Order By TotalDeathCount desc

BREAKING THINGS DOWN BY CONTINENT

In [ ]:
-- Showing Continents with the Highest Death Count per Population

Select continent, MAX(cast(Total_deaths as int)) as TotalDeathCount
From CovidDeaths
Where continent is not null
Group By continent
Order By TotalDeathCount desc

In [ ]:
-- Global Numbers

Select SUM(new_cases) as total_cases, SUM(cast(new_deaths as int)) as total_deaths, SUM(cast(new_deaths as int))/SUM(New_Cases)*100 as DeathPercentage
From CovidDeaths
Where continent is not null 
Order By 1,2

In [ ]:
-- Total Population vs. Vaccinations
-- Shows Percentage of Population that has Received at Least One COVID VACCINATION

Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
From CovidDeaths dea
Join CovidVaccinations vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 
order by 2,3

In [ ]:
-- Using CTE to Perform Calculation on Partition By in Previous Query

With PopvsVac (Continent, Location, Date, Population, New_Vaccinations, RollingPeopleVaccinated)
as
(
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
From CovidDeaths dea
Join CovidVaccinations vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null
)
Select *, (RollingPeopleVaccinated/Population)*100
From PopvsVac

In [ ]:
-- Using Temp Table to Perform Calculation on Partition By in Previous Query

DROP Table if exists #PercentPopulationVaccinated
Create Table #PercentPopulationVaccinated
(
Continent nvarchar(255),
Location nvarchar(255),
Date datetime,
Population numeric,
New_vaccinations numeric,
RollingPeopleVaccinated numeric
)

Insert into #PercentPopulationVaccinated
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
From CovidDeaths dea
Join CovidVaccinations vac
	On dea.location = vac.location
	and dea.date = vac.date
Select *, (RollingPeopleVaccinated/Population)*100
From #PercentPopulationVaccinated

In [ ]:
-- Creating View to Store Data for Later Visualizations

Create View PercentPopulationVaccinated as
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
From CovidDeaths dea
Join CovidVaccinations vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 